# Przygotowanie danych tekstowych

W poniższym notatniku skupimy się głównie na przygotowaniu danych tekstowych - ich wektoryzacji, ekstrakcji cechy i czyszczeniu. Następnie przejdziemy do budowy modelu regresji logistycznej.

In [1]:
#wczytanie podstawowych pakietów
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv') #wczytanie zbioru danych

In [3]:
train.drop(axis = 1, labels = "id", inplace = True) #id komentarzy nie będą nam potrzebne w dalszej klasyfikacji

In [4]:
train.head() 

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
#podział na zmienne objaśniające i objaśniane
X_train = train.comment_text
y_train = train.drop(axis = 1, labels = "comment_text")

In [6]:
X_train 

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [7]:
y_train

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


##  Stop words, normalizacja, stemming, lematyzacja

In [8]:
import tensorflow as tf
import keras
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score , accuracy_score , confusion_matrix , f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

### Normalizacja
Normalizacja tekstu polega na takim jego przetworzeniu, aby miał spójną formę, która ułatwi dalszą interpretację tekstu (przykłady: zmiana liter na małe bądź wielkie, rozwinięcie skrótów, normalizacja skrótowców, konwersja wyrażeń numerycznych i wyrażeń słowno-numerycznych do postaci słownej, normalizacja znaków specjalnych – takich jak symbol akapitu czy znak zastrzeżenia prawa autorskiego, usunięcie lub zmiana znaków interpunkcyjnych itd.).


In [9]:
#kod do normalizacji


In [10]:
def  clean_text(text):
    text =  text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation)) 
    text = re.sub("(\\W)"," ",text) 
    text = re.sub('\S*\d\S*\s*','', text)
    
    return text

In [11]:
X_train = X_train.apply(clean_text)

In [12]:
X_train[:10]

0    explanation why the edits made under my userna...
1    daww he matches this background colour i am se...
2    hey man i am really not trying to edit war it ...
3     more i cannot make any real suggestions on im...
4    you sir are my hero any chance you remember wh...
5      congratulations from me as well use the tool...
6         cocksucker before you piss around on my work
7    your vandalism to the matt shirvington article...
8    sorry if the word nonsense was offensive to yo...
9    alignment on this subject and which are contra...
Name: comment_text, dtype: object

### Stop words
Są to najczęściej występujące słowa języka, które na ogół nie niosą ze sobą żadnych istotnych treści. Są zatem zazwyczaj usuwane w celu optymalizacji modelu.

W celu wykrycia i usunięcia wspomnianych słów skorzystamy z biblioteki *spaCy*, a konkretnie z anglojęzycznego pakietu *STOP_WORDS*.

In [194]:
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

#  "nlp" Object is used to create documents with linguistic annotations.
def stop_words_remove(text):
    nlp_doc = nlp(text)

    #tworzymi listę tokenów z przetwarzanego tekstu
    tokens = []
    for token in nlp_doc:
        tokens.append(token.text)

    # usuwamy 'stop words' i tworzymy nową listę tokenów
    filtered_tokens =[] 

    for word in tokens:
        word_id = nlp.vocab[word]
        if word_id.is_stop == False:
            filtered_tokens.append(word)
    
    filtered_comment = ' '.join(filtered_tokens) 
    
    return filtered_comment

**Przykład działania**

In [193]:
X_train[8]

'sorry if the word nonsense was offensive to you anyway i am not intending to write anything in the articlewow they would jump on me for vandalism i am merely requesting that it be more encyclopedic so one can use it for school as a reference i have been to the selective breeding page but it is almost a stub it points to animal breeding which is a short messy article that gives you no info there must be someone around with expertise in eugenics '

In [195]:
stop_words_remove(X_train[8])

'sorry word nonsense offensive intending write articlewow jump vandalism merely requesting encyclopedic use school reference selective breeding page stub points animal breeding short messy article gives info expertise eugenics'

**Usunięcie '*stop words*' ze zbioru treningowego**

In [201]:
X_train_stop = X_train.apply(stop_words_remove)
X_train_stop

0         explanation edits username hardcore metallica ...
1         daww matches background colour seemingly stuck...
2         hey man trying edit war guy constantly removin...
3           real suggestions improvement   wondered sect...
4                             sir hero chance remember page
                                ...                        
159566    second time asking view completely contradicts...
159567                ashamed    horrible thing talk page  
159568    spitzer    umm s actual article prostitution r...
159569      looks like actually speedy version deleted look
159570        think understand   came idea bad right awa...
Name: comment_text, Length: 159571, dtype: object

### Stemming 
Stemming jest procesem usunięcia końcówki fleksyjnej ze słowa, w  czego efekcie pozostaje tylko temat wyrazu. 


In [16]:
#nltk.download('stopwords')
sn = SnowballStemmer(language='english')


def stemmer(text):
    words =  text.split()
    train = [sn.stem(word) for word in words if not word in set(stopwords.words('english'))]
    return ' '.join(train)

In [18]:
X_train_lem = X_train.apply(stemmer)

In [ ]:
X_train_stem = pd.DataFrame(X_train2)
X_train_stem.to_csv('X_train_stem.csv') 

### Lematyzacja

Lematyzacja to sprowadzenie słowa do jego podstawowej postaci. Na przykład w przypadku czasownika to najczęściej będzie bezokolicznik, w przypadku rzeczownika sprowadzamy do mianownika liczby pojedynczej.

In [51]:
import spacy


#budujemy model
#NER - Named Entity Recognition - wyłączamy
#'parser' daje informacje składniowe - póki co ich nie potrzebujemy. 
# https://spacy.io/usage/linguistic-features#disabling

# Wyłączenie parsera sprawi, że SpaCy będzie ładował się i działał znacznie szybciej
# https://spacy.io/usage/linguistic-features#named-entities
load_model = spacy.load('en_core_web_sm', disable = ['parser','ner'])

def lemmatization(text):
    text_model = load_model(text)
    result = " ".join([token.lemma_ for token in text_model])
    return result

In [54]:
X_train_lem = X_train.apply(lemmatization)

**Przykład działania**

In [57]:
X_train[0]

'explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired '

In [56]:
X_train_lem[0]

'explanation why the edit make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after I vote at new york doll fac and please do not remove the template from the talk page since I be retire'

In [58]:
X_train_lemm = pd.DataFrame(X_train_lem)
X_train_lemm.to_csv('X_train_lem.csv') 

## Wektoryzacja

Na początku przedstawimy kilka podstawowych pojęć związanych z przetwarzaniem języka naturalnego.
### „Bag of words” 
Aby algorytm mógł sobie poradzić z tekstem, musimy najpierw podzielić ten tekst na mniejsze fragmenty. Stworzenie tzw. "worka słów" jest jednym ze sposób na uzyskanie takiego podziału. Każde słowo użyte w tekście zostaje wyodrębnione i wrzucone do multizbioru. Dla przykładu, jeśli mamy dwa zdania: „Marcin ma kota.” oraz „Patryk ma psa.”, to w worku słów znajdzie się pięć słów: Marcin, ma, kota, Patryk, psa. Ich kolejność nie będzie odgrywała roli.


### N_gram
W naszym worku mogą się znaleźć nietylko pojedyńcze słowa, ale pewne sekwencje słów. N-gram jest ciągiem elementów z danej próbki tekstu bądź mowy. Zazwyczaj jednym elementem jest pojedyncze słowo (ale w określonych przypadkach mogą też być to fonemy, litery lub sylaby).

### Wektoryzacja

Z "bag of words" blisko związanym terminem jest wektoryzacjia. Najprostszym wektoryzatorem jest CountVectorizer. Zlicza on liczbę wystąpień każdego wyrazu (lub n_gramu) w tekście i przedstawia za pomocą wektora składającego się z liczb naturalnych. Każda liczba informuje, ile razy dany element wystąpił w analizowanym tekście.

!!! do zrobienia notatka czym się różni count od tf-idf, dlaczego będziemy go używać, możemy też zrobić testy jak sobie poradził count, ale to chyba przesada by była xd

https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a




In [ ]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode', #normalizacja tekstu, usuwanie akcentów itp. unicode jest wolniejsza, ale radzi sobie z dowolnymi znakami   
    token_pattern=r'\w{1,}',  #co zaliczamy jako token - tutaj są to obiekty typu r'\w' czyli o kategorii alfabetonumerycznej, o długości 1 lub większej   
    ngram_range=(1, 3),      #liczba możliwych n-gramów - tutaj dopuszczamy mono-, bi-, i tri-gramy  
    stop_words='english', #jaka kategoria dla stopwords, domyślnie jest None, dostępna jest opcja 'english' lub inna własna lista
    sublinear_tf=True) #zamiast term frequency (tf) oddaje 1+ln(tf)

word_vectorizer.fit(X_train)    
train_word_features = word_vectorizer.transform(X_train)

In [ ]:
X_train_transformed = word_vectorizer.transform(x_train)

In [ ]:
print(X_train_transformed)

źródła: https://www.statystyczny.pl/klasyfikacja-tekstu-text-classification/

### Wektoryzacja - tensor ze zmiennymi binarnymi

Alternatywnym sposobem wektoryzacji będzie stworzenie tensora ze zmiennymi zero-jedynkowymi. Każda z nich odpowiada za wystąpienie bądź niewystąpienie danego słowa w danym komentarzu. Postać binarna macierzy powinna zapobiec problemowi zbyt dużej złożoności obliczeniowej.

Jednak żeby uniknąć macierzy o zbyt dużych wymiarach wybierzemy jedynie 1000 najczęściej występujących słów w zbiorze. 

In [203]:
X_train_bin_vect = X_train_lem.apply(stop_words_remove)
X_train_bin_vect

0         explanation edit username hardcore metallica f...
1         daww match background colour seemingly stuck t...
2         hey man try edit war guy constantly remove rel...
3            real suggestion improvement    wonder secti...
4                             sir hero chance remember page
                                ...                        
159566    second time ask view completely contradict cov...
159567               ashamed     horrible thing talk page  
159568    spitzer     umm s actual article prostitution ...
159569        look like actually speedy version delete look
159570          think understand    come idea bad right ...
Name: comment_text, Length: 159571, dtype: object

In [204]:
full_text = ' '.join(list(X_train_bin_vect))
full_text = full_text.split()
words = pd.DataFrame(full_text).value_counts()

In [214]:
words_5000 = words.rename_axis(['word'])\
            .reset_index()\
            .rename(columns={0:'count'})\
            .sort_values(by=['count'], ascending=False)

print(words_5000.head())
words_sequence = words_5000.loc[0:5000,'word'].values
words_sequence

        word  count
0    article  72932
1       page  56447
2  wikipedia  37072
3       edit  36560
4       talk  33962


array(['article', 'page', 'wikipedia', ..., 'resonance', 'intact',
       'physician'], dtype=object)

In [231]:
def vectorize_sequences(dataset, sequences, dimension=1000):
    results = np.zeros(shape = (dataset.shape[0], dimension), dtype=int)
    for j in range(dataset.shape[0]):
        for i,sequence in enumerate(sequences[0:1000]):
            if sequence in dataset[j]:
                results[j,i] = 1
                
    return results

In [235]:
# chyba jednak zbyt duza ta tablica
#X_train_vect_bin = vectorize_sequences(X_train_bin_vect, list(words_sequence))

# Model

In [ ]:
log_reg = LogisticRegression(C = 10, penalty='l2', solver = 'liblinear', random_state=45)

classifier = OneVsRestClassifier(log_reg)
classifier.fit(X_train_transformed, y_train)


y_train_pred_proba = classifier.predict_proba(X_train_transformed)
y_test_pred_proba = classifier.predict_proba(X_test_transformed)


roc_auc_score_train = roc_auc_score(y_train, y_train_pred_proba,average='weighted')
roc_auc_score_test = roc_auc_score(y_test, y_test_pred_proba,average='weighted')

print("ROC AUC Score Train:", roc_auc_score_train)
print("ROC AUC Score Test:", roc_auc_score_test)

In [ ]:
def make_test_predictions(df,classifier):
    df.comment_text = df.comment_text.apply(clean_text)
    df.comment_text = df.comment_text.apply(stemmer)
    X_test = df.comment_text
    X_test_transformed = vectorize.transform(X_test)
    y_test_pred = classifier.predict_proba(X_test_transformed)
    return y_test_pred

In [ ]:
xx ={'id':[565],'comment_text':['Steve is Steve is Steve']}
xx = pd.DataFrame(xx)
make_test_predictions(xx,classifier)